In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data.csv',index_col=0)

In [3]:
import gensim.models

In [4]:
model = gensim.models.Word2Vec.load('word2vec.fullmodel')

In [5]:
len(model.wv)

211464

In [6]:
captions=list(df['caption'])

In [7]:
from gensim.parsing.preprocessing import remove_stopwords, preprocess_string,strip_non_alphanum,strip_numeric

In [20]:
import numpy as np

In [21]:
def caption_2_represention(captionfile):
    size=300
    for line in open(f'{captionfile}'):
        CUSTOM_FILTERS = [lambda x: x.lower(), remove_stopwords, strip_non_alphanum,strip_numeric]
        tokens=preprocess_string(line,CUSTOM_FILTERS)
        tokens_filtered_in_vocab = [token for token in tokens if token in model.wv.index_to_key] 
        embedding = np.zeros(size)
        for tok in tokens_filtered_in_vocab:
            embedding += model.wv[tok]
            
        if len(tokens_filtered_in_vocab) > 1:
            embedding /= len(tokens_filtered_in_vocab)
            
        if min(embedding) < 0:
            embedding = embedding - min(embedding)
        if max(embedding) > 0:
            embedding = embedding / max(embedding)
        out_string = ''
        for t in range(0,300):
            out_string = out_string + ','+ str(embedding[t])
        return out_string

In [22]:
from pandarallel import pandarallel
pandarallel.initialize()

INFO: Pandarallel will run on 64 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [23]:
train_df = pd.read_csv('train.csv')

In [17]:
train_df.head()

,caption,img,label,caption_embedding
0,captions_resized_1M/cities_instagram/chicago/1...,img_resized_1M/cities_instagram/chicago/148202...,chicago,",0.45350155971180706,0.3800610459612754,0.3849..."
1,captions_resized_1M/cities_instagram/miami/148...,img_resized_1M/cities_instagram/miami/14898629...,miami,",0.45350155971180706,0.3800610459612754,0.3849..."
2,captions_resized_1M/cities_instagram/london/14...,img_resized_1M/cities_instagram/london/1490670...,london,",0.45350155971180706,0.3800610459612754,0.3849..."
3,captions_resized_1M/cities_instagram/losangele...,img_resized_1M/cities_instagram/losangeles/149...,losangeles,",0.45350155971180706,0.3800610459612754,0.3849..."
4,captions_resized_1M/cities_instagram/sydney/14...,img_resized_1M/cities_instagram/sydney/1480253...,sydney,",0.45350155971180706,0.3800610459612754,0.3849..."


In [18]:
train_df.columns

Index(['caption', 'img', 'label', 'caption_embedding'], dtype='object')

In [24]:
train_df['caption_embedding'] = train_df['caption'].parallel_apply(caption_2_represention)test

In [25]:
train_df.head()

,caption,img,label,caption_embedding
0,captions_resized_1M/cities_instagram/chicago/1...,img_resized_1M/cities_instagram/chicago/148202...,chicago,",0.38072692005564585,0.482761029263083,0.42480..."
1,captions_resized_1M/cities_instagram/miami/148...,img_resized_1M/cities_instagram/miami/14898629...,miami,",0.4431235275782183,0.45451776316361425,0.3193..."
2,captions_resized_1M/cities_instagram/london/14...,img_resized_1M/cities_instagram/london/1490670...,london,",0.33210418794562097,0.38874253433663797,0.549..."
3,captions_resized_1M/cities_instagram/losangele...,img_resized_1M/cities_instagram/losangeles/149...,losangeles,",0.4553817188828261,0.38173804270409417,0.0820..."
4,captions_resized_1M/cities_instagram/sydney/14...,img_resized_1M/cities_instagram/sydney/1480253...,sydney,",0.28686858187748865,0.5030648779167651,0.5735..."


In [26]:
train_df.to_csv('train.csv',index=False)

In [27]:
test_df = pd.read_csv('test.csv')

In [28]:
test_df['caption_embedding'] = test_df['caption'].parallel_apply(caption_2_represention)

In [30]:
test_df.to_csv('test.csv',index=False)

In [31]:
val_df = pd.read_csv('val.csv')

In [32]:
val_df['caption_embedding'] = val_df['caption'].parallel_apply(caption_2_represention)

In [33]:
val_df.to_csv('val.csv',index=False)

In [35]:
val_df.head()

,caption,img,label,caption_embedding
0,captions_resized_1M/cities_instagram/chicago/1...,img_resized_1M/cities_instagram/chicago/149533...,chicago,",0.6664006476095762,0.4473309227578546,0.48012..."
1,captions_resized_1M/cities_instagram/sanfranci...,img_resized_1M/cities_instagram/sanfrancisco/1...,sanfrancisco,",0.3293704904514245,0.5152873409369172,0.51827..."
2,captions_resized_1M/cities_instagram/chicago/1...,img_resized_1M/cities_instagram/chicago/148201...,chicago,",0.2664918577084412,0.6018341612634897,0.67461..."
3,captions_resized_1M/cities_instagram/sanfranci...,img_resized_1M/cities_instagram/sanfrancisco/1...,sanfrancisco,",0.3824586303841255,0.46444768090201566,0.6064..."
4,captions_resized_1M/cities_instagram/sydney/14...,img_resized_1M/cities_instagram/sydney/1487239...,sydney,",0.41056553851094957,0.5116613575150174,0.5365..."
